In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

def makeDf(directory, nFreqs):
        #read LPV
        df = pd.read_csv(directory+'/LPV.txt',
                skiprows=5,
                header=None, 
                names=['X', 'Y', 'Z', 'Freq', 'LPV'],
                delimiter=' ',
                skipinitialspace=True)

        E_near_x = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=4)
        E_near_y = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=5)
        E_near_z = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=6)
        E_near_norm = np.loadtxt(directory+'/avgE_NearField.txt', skiprows=5, usecols=7)

        E_all_x = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=4)
        E_all_y = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=5)
        E_all_z = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=6)
        E_all_norm = np.loadtxt(directory+'/avgE_all_64pts.txt', skiprows=5, usecols=7)

        nPoints = df.shape[0]
        nAntPositions = nPoints//nFreqs
        positions = [x for x in range(nAntPositions)]
        posList = np.repeat(positions, nFreqs)
        freqs = np.asarray(df['Freq'][0:nFreqs])

        df['Position'] = posList
#        cols = df.columns.tolist()
#        cols = cols[-1:] + cols[:-1]
#        df = df[cols]

        df['E_near_X'] = E_near_x
        df['E_near_Y'] = E_near_y
        df['E_near_Z'] = E_near_z
        df['E_near_Norm'] = E_near_norm
        df['E_all_X'] = E_all_x
        df['E_all_Y'] = E_all_y
        df['E_all_Z'] = E_all_z
        df['E_all_Norm'] = E_all_norm

        xpos = np.asarray(df.groupby('Position').mean()['X'])
        ypos = np.asarray(df.groupby('Position').mean()['Y'])
        zpos = np.asarray(df.groupby('Position').mean()['Z'])


        #set order of columns
        cols = ['Position',
                'Freq', 
                'LPV',
                'E_near_Norm',
                'E_all_Norm',
                'E_near_X',
                'E_near_Y',
                'E_near_Z',
                'E_all_X',
                'E_all_Y',
                'E_all_Z',
                'X',
                'Y',
                'Z']
        df = df[cols]

        #Print info
        print(directory)
        print(nAntPositions, " Antenna Positions :")
        print(np.asarray([positions,xpos, ypos, zpos, ]).T)
        print('Freq (start stop number step) :', freqs[0], ',', freqs[-1],',', len(freqs),',', freqs[1]-freqs[0])
        print('\n')
        return df, freqs, nAntPositions



In [ ]:
plt.close('all')
%matplotlib widget

plt.figure()
plt.title('Port Voltage')
for i in range(nAntPositions):
    plt.plot(freqs, df[df['Position']==i]['LPV'], alpha = .7)
plt.plot(freqs, df.groupby('Freq').mean()['LPV'],'r--',label = 'Mean')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Port Voltage (Volts)')
plt.legend()

plt.figure()
plt.title('AF (E = 1V/m)')
for i in range(nAntPositions):
    plt.plot(freqs, 20*np.log10(1/df[df['Position']==i]['LPV']), alpha = .7, label = '%i'%i)
plt.plot(freqs, 20*np.log10(1/df.groupby('Freq').mean()['LPV']),'r--',label = 'Mean')
plt.xlabel('Frequency (MHz)')
plt.ylabel('20log10(1/V) ')
plt.legend()

plt.figure()
plt.title('AF (E = 1V/m)')
for i in range(nAntPositions):
    plt.plot(freqs, 20*np.log10(df[df['Position']==i]['E_near_Norm']/df[df['Position']==i]['LPV']), alpha = .7, label = '%i'%i)
plt.plot(freqs, 20*np.log10(1/df.groupby('Freq').mean()['LPV']),'r--',label = 'Mean')
plt.xlabel('Frequency (MHz)')
plt.ylabel('20log10(1/V) ')
plt.legend()

plt.figure()
plt.title('E Averaged Around Antenna')
for i in range(nAntPositions):
    plt.plot(freqs, df[df['Position']==i]['E_near_Norm'], alpha = .7)
plt.plot(freqs, df.groupby('Freq').mean()['E_near_Norm'],'r--',label = 'Mean')
plt.xlabel('Frequency (MHz)')
plt.ylabel('E(V/m)')
plt.legend()

plt.figure()
plt.title('E Averaged Over Room')
for i in range(nAntPositions):
    plt.plot(freqs, df[df['Position']==i]['E_all_Norm'], alpha = .7)
plt.plot(freqs, df.groupby('Freq').mean()['E_all_Norm'],'r--',label = 'Mean')
plt.xlabel('Frequency (MHz)')
plt.ylabel('E(V/m)')
plt.legend()

In [ ]:
#compare awe to non awe
nFreqs = 80
AWE_df, freqs, nAntPositions = makeDf('1pos_98_106MHz_80freqpts_cons5.9e6_perm1000_realDimentions_AWE', nFreqs=nFreqs)
nonAWE_df, freqs, nAntPositions = makeDf('1pos_98_106MHz_80freqpts_cons5.9e6_perm1000_realDimentions_nonAWE', nFreqs=nFreqs)

plt.figure()
plt.title('Port Voltage')
plt.scatter(freqs, AWE_df.groupby('Freq').mean()['LPV'],label = 'AWE')
plt.plot(freqs, AWE_df.groupby('Freq').mean()['LPV'],label = 'AWE')
plt.scatter(freqs, nonAWE_df.groupby('Freq').mean()['LPV'],label = 'Non AWE')
plt.plot(freqs, nonAWE_df.groupby('Freq').mean()['LPV'],label = 'Non AWE')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Port Voltage (Volts)')
plt.legend()



In [ ]:
#generate names
avgList = ['near', 'all']
vecList = ['X', 'Y', 'Z', 'Norm']
for avgType in avgList:
    for vecType in vecList:
        print("'E_"+avgType+"_"+vecType+"',")
        #print("df['E_"+avgType+"_"+vecType+"'] = E_"+avgType.lower()+"_"+vecType.lower()) 

In [ ]:

x = 3.0697 #comsol width, real E(ast)
y = 2.4566 #comsel depth, real V(ert)
z = 3.6840 #comsol height, real N(orth)
 
def comsolToReal(xIn = -x/2, yIn = -y/2 , zIn = -z/2):
    print('North = ', z/2 - zIn)
    print('East = ', xIn + x/2)
    print('Vertical = ', yIn + y/2)


def realToComsol(nIn = -z/2, eIn = x/2, vIn = y/2):
    print('X =', eIn - x/2)
    print('Y =', vIn - y/2)
    print('Z =', nIn + z/2)

realToComsol()


In [ ]:
randList = [np.random.randint(90) for x in range(18)]
randList

In [ ]:
bigdf, freqs, nAntPositions = makeDf('18pos_50_300MHz_500freqpts_cons5.9e6_perm1000_realDimentions_nonAWE', 500)
bigdfAwe, freqs, nAntPositions = makeDf('18pos_50_300MHz_500freqpts_cons5.9e6_perm1000_realDimentions_AWE', 500)

plt.close('all')
%matplotlib widget

plt.figure()
plt.title('Port Voltage')
if 1:
    for i in range(nAntPositions):
        plt.plot(freqs, (bigdf[bigdf['Position']==i]['LPV']), alpha = .7)

#plt.plot(freqs, 20*np.log10(bigdf.groupby('Freq').median()['LPV']),'r',label = 'non AWE')
#plt.plot(freqs, 20*np.log10(bigdf.groupby('Freq').mean()['LPV']),label = 'mean')
#plt.plot(freqs, 20*np.log10(bigdfAwe.groupby('Freq').mean()['LPV']) ,label = 'AWE', alpha = .6)
plt.xlabel('Frequency (MHz)')
plt.ylabel('20 log10(Port Voltage)')
plt.legend()

In [ ]:
df, freqs, nAntPositions = makeDf('18pos_118_131MHz_250freqpts_cons5.9e6_perm1000_realDimentions_nonAWE', 250)

plt.close('all')
%matplotlib widget

plt.figure()
plt.title('Port Voltage')
if 1:
    for i in range(nAntPositions):
        plt.plot(freqs, 20*np.log10(df[df['Position']==i]['LPV']),
        alpha = .7,
        label = '%i'%i)

plt.plot(freqs, 20*np.log10(df.groupby('Freq').mean()['LPV']),'r',label = 'non AWE')

if 1:
    posNum = 6
    plt.plot(freqs, 20*np.log10(df[df['Position']==posNum]['LPV']),
        'r--',
        alpha = 1,
        label = '%i'%posNum)
plt.xlabel('Frequency (MHz)')
plt.ylabel('20 log10(Port Voltage)')
plt.legend()

In [ ]:
df.keys()

In [ ]:
df